# Atividade Enzimática

In [ ]:
import pandas as pd
import dataframe_image as dfi
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from lmfit import Model

In [ ]:
# importanto o dataframe

# Código para ler o arquivo em xls e retorná-lo em dataframe
df_AMP = pd.read_excel("1. Dados-curva-padrao-AMP (1).xlsx")

df_Km_Ala = pd.read_excel("2. Dados-curva-Km-Ala (1).xlsx")

df_ATP = pd.read_excel("3. Dados-cinética-ATP (1).xlsx")

In [ ]:
df_AMP.head()

In [ ]:
df_Km_Ala.head()

In [ ]:
df_ATP.head()

## Atividade 1

In [ ]:
df_AMP["Média ABS"] = (df_AMP["Replica 1"] + df_AMP["Replica 2"] + df_AMP["Replica 3"] + df_AMP["Replica 4"])/4

branco = df_AMP.loc[7]["Média ABS"]

df_AMP["Média ABS sem branco"] = df_AMP["Média ABS"] - branco

In [ ]:
df_AMP.head()

In [ ]:
#df_styled = df_AMP.style.background_gradient()
#dfi.export(df_styled, "df_AMP.png")

In [ ]:
x = np.array(list(df_AMP["Concentração AMP (µM)"])).reshape((-1, 1))
y = np.array(list(df_AMP["Média ABS sem branco"]))

model = LinearRegression()

model.fit(x, y)

model = LinearRegression().fit(x, y)

r_sq = model.score(x, y)
print(f"coefficient of determination: {r_sq}")

print(f"intercept: {model.intercept_}")

print(f"slope: {model.coef_}")

a = model.coef_[0]

b = model.intercept_

In [ ]:
def ABS(AMP):
    return b + a*AMP

AMP = np.linspace(0, 25, 100)

fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
plt.plot(AMP, ABS(AMP), color='black')
df_AMP.plot('Concentração AMP (µM)','Média ABS sem branco',legend=True, ax=ax1, kind = 'scatter', linewidth = 2.5, color = 'blue')
#plt.title('Curva de calibração do AMP')
plt.xlabel('[AMP] (µM)')
plt.ylabel('RFU')
plt.savefig('reg_AMP.png', dpi=100, format='png', facecolor='white')
plt.show()

In [ ]:
a

## Atividade 2

In [ ]:
df_Km_Ala

In [ ]:
#df_styled = df_Km_Ala.style.background_gradient()
#dfi.export(df_styled, "df_Km_Ala_sem_mudancas.png")

In [ ]:
df_Km_Ala["Média v"] = (df_Km_Ala["v1"] + df_Km_Ala["v2"] + df_Km_Ala["v3"] + df_Km_Ala["v4"])/4
df_Km_Ala["Média v"] = df_Km_Ala["Média v"] - df_Km_Ala.loc[7]["Média v"]
df_Km_Ala = df_Km_Ala.drop(7)
df_Km_Ala["inverso Média v"] = 1/df_Km_Ala["Média v"]
df_Km_Ala["inverso [L-Ala] (mM)"] = 1/df_Km_Ala["[L-Ala] (mM)"]

In [ ]:
df_Km_Ala

In [ ]:
x = np.array(list(df_Km_Ala["inverso [L-Ala] (mM)"])).reshape((-1, 1))
y = np.array(list(df_Km_Ala["inverso Média v"]))

model = LinearRegression()

model.fit(x, y)

model = LinearRegression().fit(x, y)

r_sq = model.score(x, y)
print(f"coefficient of determination: {r_sq}")

print(f"intercept: {model.intercept_}")

print(f"slope: {model.coef_}")

a = model.coef_[0]

b = model.intercept_

V_max = 1/b

K = V_max*a

In [ ]:
def inv_v(inv_Ala):
    return b + a*inv_Ala

inv_Ala = np.linspace(0, 0.128, 100)

fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
plt.plot(inv_Ala, inv_v(inv_Ala), color='black')
df_Km_Ala.plot('inverso [L-Ala] (mM)','inverso Média v',legend=True, ax=ax1, kind = 'scatter', linewidth = 2.5, color = 'blue')
plt.xlabel('1/[L-Ala] (mM)')
plt.ylabel('1/v')
plt.savefig('reg_aanc.png', dpi=100, format='png', facecolor='white')
plt.show()

In [ ]:
print(V_max, K)

In [ ]:
def v(Ala):
    return V_max*Ala/(K + Ala)

Ala = np.linspace(0, 500, 500)

fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
plt.plot(Ala, v(Ala), color='black')
df_Km_Ala.plot('[L-Ala] (mM)','Média v',legend=True, ax=ax1, kind = 'scatter', linewidth = 2.5, color = 'blue')
plt.xlabel('[L-Ala] (mM)')
plt.ylabel('v')
plt.savefig('mod_aanc_dados.png', dpi=100, format='png', facecolor='white')
plt.show()

In [ ]:
def v_2(Ala, V_max_2, K_m_2):
    return V_max_2*(Ala)/(K_m_2 + Ala)

In [ ]:
modelo_Ala = Model(v_2)

print(f'Variável independente: {modelo_ATP.independent_vars}')
print(f'Parâmetros do modelo: {modelo_ATP.param_names}')

In [ ]:
params = modelo_Ala.make_params(V_max_2=7000, K_m_2 = 125)

In [ ]:
resultado_fit = modelo_Ala.fit(df_Km_Ala["Média v"], params, Ala=df_Km_Ala["[L-Ala] (mM)"])
print(resultado_fit.fit_report())

In [ ]:
plt.plot(df_Km_Ala["[L-Ala] (mM)"], df_Km_Ala["Média v"], 'o')
plt.plot(df_Km_Ala["[L-Ala] (mM)"], resultado_fit.init_fit, '--', label='Palpite inicial')
plt.plot(df_Km_Ala["[L-Ala] (mM)"], resultado_fit.best_fit, '-', label='Melhor ajuste encontrado')
plt.legend()
#plt.ylim(0, 1000)
#plt.yscale('log')
plt.show()

In [ ]:
resultado_fit.plot()
#plt.savefig('aanc_lmfit.png', dpi=100, format='png', facecolor='white')
plt.show()

In [ ]:
V_max = 10412.66
K = 253.87

def v(Ala):
    return V_max*Ala/(K + Ala)

Ala = np.linspace(0, 500, 500)

fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
plt.plot(Ala, v(Ala), color='black')
df_Km_Ala.plot('[L-Ala] (mM)','Média v',legend=True, ax=ax1, kind = 'scatter', linewidth = 2.5, color = 'blue')
plt.xlabel('[L-Ala] (mM)')
plt.ylabel('v')
plt.savefig('aanc_lmfit.png', dpi=100, format='png', facecolor='white')
plt.show()

## Atividade 3

In [ ]:
df_ATP["Média v"] = (df_ATP["v1"] + df_ATP["v2"] + df_ATP["v3"] + df_ATP["v4"])/4
df_ATP["Média v"] = df_ATP["Média v"] - df_ATP["Blank"]

In [ ]:
def v_2(ATP, V_max_2, K_prime, h):
    return V_max_2*(ATP**h)/(K_prime + ATP**h)

In [ ]:
modelo_ATP = Model(v_2)

print(f'Variável independente: {modelo_ATP.independent_vars}')
print(f'Parâmetros do modelo: {modelo_ATP.param_names}')

In [ ]:
params = modelo_ATP.make_params(V_max_2=5000, K_prime = 1, h = 1.1)

In [ ]:
resultado_fit = modelo_ATP.fit(df_ATP["Média v"], params, ATP=df_ATP["[ATP] (mM)"])
print(resultado_fit.fit_report())

In [ ]:
plt.plot(df_ATP["[ATP] (mM)"], df_ATP["Média v"], 'o')
plt.plot(df_ATP["[ATP] (mM)"], resultado_fit.init_fit, '--', label='Palpite inicial')
plt.plot(df_ATP["[ATP] (mM)"], resultado_fit.best_fit, '-', label='Melhor ajuste encontrado')
plt.legend()
#plt.ylim(0, 1000)
#plt.yscale('log')
plt.show()

In [ ]:
resultado_fit.plot()
plt.savefig('Modelo da reação enzimática.png', dpi=100, format='png', facecolor='white')
plt.show()

In [ ]:
V_max = 4911.27
K = 0.046
h = 3.18

def v(ATP):
    return V_max*(ATP**h)/(K + (ATP**h))

ATP = np.linspace(0, 2, 500)

fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
plt.plot(ATP, v(ATP), color='black')
df_ATP.plot('[ATP] (mM)',"Média v",legend=True, ax=ax1, kind = 'scatter', linewidth = 2.5, color = 'blue')
plt.xlabel('[ATP] (mM)')
plt.ylabel('v')
plt.savefig('ATP_lmfit.png', dpi=100, format='png', facecolor='white')
plt.show()